In [3]:
To do:
    can everyone use curl to download the index?
    
    show how to set up notebook in R before opening this
    
    https://docs.anaconda.com/anaconda/navigator/tutorials/r-lang/

SyntaxError: invalid syntax (<ipython-input-3-04b27588e5c3>, line 1)

# Downloading the fastq files

Make a new directory on your computer for todays project.

On my laptop I am using:

    "/Users/alexis/Desktop/bd2k/"
    
Inside of the BD2K folder I will make a folder to save the fastq files.

    "/Users/alexis/Desktop/bd2k/fastq"
    
????? download by ?????????

condition: samples
    
euploid:

    SRR5494631    SRR5494630
    
aneuploid:

    SRR5494629    SRR5494628 

# Running kallisto

Since everyone should now have anaconda installed you install kallisto by running...

`conda install kallisto`


https://pachterlab.github.io/kallisto/download

#### Building the index

Before we run kallisto we need to build the index. Download the transcriptome using...

`curl -O ftp://ftp.ensembl.org/pub/release-94/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz`


    Usage: kallisto index [arguments] FASTA-files

    Required argument:
    -i, --index=STRING          Filename for the kallisto index to be constructed 

    
`kallisto index -i 	Homo_sapiens.GRCh38.cdna.all.release-94_k31.idx	Homo_sapiens.GRCh38.cdna.all.fa.gz`


More help for building the index: https://github.com/pachterlab/kallisto-transcriptome-indices

#### Running kallisto
There are a total of 4 samples and you will need to run kallisto on each sample individually.

    Usage: kallisto quant [arguments] FASTQ-files

    arguments:
    -i, --index=STRING            Filename for the kallisto index to be used for
                              quantification
                              
    -o, --output-dir=STRING       Directory to write output to


    -b, --bootstrap-samples=INT   Number of bootstrap samples (default: 0)


##### euploid samples

`kallisto quant -i Homo_sapiens.GRCh38.cdna.all_transcripts.idx -o /path/to/SRR5494631 SRR5494631.fastq -b 100 `

`kallisto quant -i Homo_sapiens.GRCh38.cdna.all_transcripts.idx -o /path/to/SRR5494630 SRR5494630.fastq -b 100 `

##### aneuploid samples

`kallisto quant -i Homo_sapiens.GRCh38.cdna.all_transcripts.idx -o /path/to/SRR5494629 SRR5494629.fastq -b 100 `

`kallisto quant -i Homo_sapiens.GRCh38.cdna.all_transcripts.idx -o /path/to/SRR5494628 SRR5494628.fastq -b 100 `





# Running sleuth

In [ ]:
Sleuth is an R library

# deseq2

In [ ]:
To run deseq2 on our sleuth outputs we need to use tximport


We also need to make a file 

        samples
1    RNF43_WT_1
2    RNF43_WT_2
3    RNF43_WT_3
4    RNF43_WT_4
5 RNF43_659fs_1
6 RNF43_659fs_2
7 RNF43_659fs_3
8 RNF43_659fs_4



In [ ]:
#https://bioconductor.org/packages/release/bioc/vignettes/tximport/inst/doc/tximport.html#kallisto

library(tximportData)
library(tximport)
library(ensembldb)
library(EnsDb.Hsapiens.v86)
library(DESeqDataSetFromTximport)
library(DESeq2)

#put the path to your directory that contains your kallisto outputs here 

dir <- "/Users/alexis/Desktop/KRASG12V-sleuth/KRAS_G12V_venn_kallisto_out_GRCh38_bootstrap100"
list.files(dir)

samples <- read.table("/Users/alexis/Desktop/KRASG12V-kallisto-deseq/samples.txt",header = TRUE)
samples

files <- file.path(dir, samples$sample, "abundance.h5")
names(files) <- samples$sample

files
all(file.exists(files))

#make conversion data frame
edb <- EnsDb.Hsapiens.v86
tx2gene = transcripts(edb , columns=c("tx_id", "gene_id"),return.type="DataFrame")
head(tx2gene)

#import 
txi.kallisto <- tximport(files, type = "kallisto",tx2gene = tx2gene, txOut = FALSE,ignoreTxVersion=TRUE,scaledTPM=TRUE)
head(txi.kallisto$counts)


sampleTable <- read.table("/Users/alexis/Desktop/KRASG12V-kallisto-deseq/samples-table.txt",header = TRUE)
sampleTable

rownames(sampleTable) <- colnames(txi.kallisto$counts)
sampleTable

dds <- DESeqDataSetFromTximport(txi.kallisto, sampleTable, ~condition)

directory <- '/Users/alexis/Desktop/KRASG12V-kallisto-deseq'

sampleTable<-data.frame(sampleName=sampleFiles, fileName=sampleFiles, condition=sampleCondition)
sampleTable

#### WT vs G12V
sampleName <- c("KRAS_G12V","KRAS_G12V","KRAS_G12V","KRAS_WT","KRAS_WT","KRAS_WT")
sampleCondition <- c("KRAS_G12V","KRAS_G12V","KRAS_G12V","KRAS_WT","KRAS_WT","KRAS_WT")

sampleTable<-data.frame(sampleName=sampleFiles, fileName=sampleFiles, condition=sampleCondition)
sampleTable

dds <- DESeqDataSetFromTximport(txi.kallisto, sampleTable, ~condition)

dds

colData(dds)$condition<-factor(colData(dds)$condition, levels=c('KRAS_WT','KRAS_G12V'))

WT_G12V_dds<-DESeq(dds)

WT_G12V_res<-results(WT_G12V_dds)
WT_G12V_res<-WT_G12V_res[order(WT_G12V_res$padj),]
head(WT_G12V_res)



## Merge with normalized count data
WT_G12V_resdata <- merge(as.data.frame(WT_G12V_res), as.data.frame(counts(WT_G12V_dds, normalized=TRUE)), by="row.names", sort=FALSE)
names(WT_G12V_resdata)[1] <- "Gene"
head(WT_G12V_resdata)
mcols(WT_G12V_res,use.names=TRUE)
write.csv(as.data.frame(WT_G12V_res),file='WT_KRAS_G12V_res_results_deseq2.csv')
write.csv(as.data.frame(WT_G12V_resdata),file='WT_KRAS_G12V_res_results_deseq2_counts.csv')


